In [ ]:
import pandas as pd
import requests


In [10]:
import requests
import io
from PIL import Image
import base64
from dash import Dash, html, dcc, Output, Input

# Initialize app
app = Dash(__name__)
server = app.server  # for deployment later if you want

# Constants
URL = "https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi"
BBOX = "-64.4373,-10.8673,-61.3161,-8.6430"

def get_image_base64(date_str):
    params = {
        "service": "WMS",
        "version": "1.1.1",
        "request": "GetMap",
        "layers": "MODIS_Combined_L3_IGBP_Land_Cover_Type_Annual",
        "styles": "",
        "format": "image/png",
        "transparent": "false",
        "srs": "EPSG:4326",
        "time": date_str,
        "bbox": BBOX,
        "width": "694",
        "height": "495",
    }
    r = requests.get(URL, params=params)
    r.raise_for_status()
    img = Image.open(io.BytesIO(r.content))
    buf = io.BytesIO()
    img.save(buf, format="PNG")
    encoded = base64.b64encode(buf.getvalue()).decode("ascii")
    return f"data:image/png;base64,{encoded}"

YEARS = list(range(2001, 2023 + 1))

app.layout = html.Div([
    html.H3("MODIS Land Cover Type (L3, IGBP, Yearly)"),
    dcc.Slider(
        min=min(YEARS),
        max=max(YEARS),
        step=1,
        value=min(YEARS),
        marks={y: str(y) for y in YEARS},
        id="year-slider"
    ),
    html.Div(id="date-label", style={"margin": "10px", "font-weight": "bold"}),
    html.Img(id="landcover-image", style={"width": "700px", "border": "1px solid #ccc"})
])

@app.callback(
    [Output("landcover-image", "src"), Output("date-label", "children")],
    [Input("year-slider", "value")]
)
def update_image(year):
    date_str = f"{year}-01-01"
    print(f"Fetching {date_str}")
    return get_image_base64(date_str), f"Year: {year}"

if __name__ == "__main__":
    app.run(debug=True)



Fetching 2001-01-01
Fetching 2003-01-01
Fetching 2004-01-01
Fetching 2005-01-01
Fetching 2006-01-01
Fetching 2014-01-01
Fetching 2020-01-01
Fetching 2001-01-01
Fetching 2018-01-01
